In [3]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv("dong_area.csv")

# 열 이름 정리
df = df.rename(columns={'동별(1)': '시', '동별(2)': '구'})

# 예외 데이터 제거 (예: '소계' 행)
df = df[~df['행정동'].str.contains("소계", na=False)]

# ▶ 전처리: '행정동'의 앞부분(예: 전농1동 → 전농) 추출
df['행정동그룹'] = df['행정동'].str.extract(r'(^[가-힣]+)')
df['마리수'] = ''  # 또는: pd.NA

# ▶ 그룹별 총 면적 계산
group_total_area = df.groupby('행정동그룹')['면적'].sum().rename('총그룹면적')

# ▶ 기존 df와 병합
df = df.merge(group_total_area, left_on='행정동그룹', right_index=True)

# ▶ 면적 비율 계산
df['면적비율(%)'] = round((df['면적'] / df['총그룹면적']) * 100, 2)

# ▶ 결과 컬럼 정리
df_result = df[['시', '구', '행정동', '면적', '면적비율(%)']]

# 저장
df_result.to_csv("dong_area_면적비율추가.csv", index=False, encoding='utf-8-sig')
print("✅ 'dong_area_면적비율추가.csv' 저장 완료!")


✅ 'dong_area_면적비율추가.csv' 저장 완료!


In [6]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv("dong_area.csv")

# '동별(2)' → '구'로 이름 바꾸기
df = df.rename(columns={'동별(2)': '구'})

# '소계' 및 NaN 제거
df = df[df['행정동'].notna() & ~df['행정동'].str.contains('소계')]

# 서대문구만 필터링
sdm_df = df[df['구'] == '서대문구'].copy()

# 면적 총합
total_area = sdm_df['면적'].sum()

# 면적 비율 계산
sdm_df['면적비율(%)'] = round((sdm_df['면적'] / total_area) * 100, 2)

# 결과 확인
print(sdm_df[['행정동', '면적', '면적비율(%)']])

# 필요 시 저장
sdm_df.to_csv("서대문구_면적비율.csv", index=False, encoding='utf-8-sig')
print("✅ '서대문구_면적비율.csv' 저장 완료!")


✅ '마리수' 열 추가 완료! 총합: 19937마리 → '서대문구_면적비율_마리수추가.csv' 저장됨


0     1097
1     1392
2     1188
3      915
4     1786
5     2329
6      576
7      871
8      622
9      949
10    1266
11     520
12    2976
13    3450
Name: 마리수, dtype: int64

In [7]:
import pandas as pd
import numpy as np

# 파일 불러오기
df = pd.read_csv("서대문구_면적비율.csv")

# 총 마리 수
total_count = 19937

# 비율 기준으로 마리수 계산 (소수점 반올림)
df['마리수계산'] = (df['면적비율(%)'] * total_count / 100).round()

# 보정 전 합계 확인
diff = int(total_count - df['마리수계산'].sum())

# 보정: 차이가 0이 아닐 경우, 차이만큼 값을 일부 행에 분배
if diff != 0:
    # 절댓값만큼 상위 n개 행 선택
    adjust_indices = df['마리수계산'].sort_values(ascending=(diff < 0)).index[:abs(diff)]
    df.loc[adjust_indices, '마리수계산'] += np.sign(diff)

# 마리수 열 확정
df['마리수'] = df['마리수계산'].astype(int)
df.drop(columns=['마리수계산'], inplace=True)

# 결과 저장
df.to_csv("서대문구_면적비율_마리수추가.csv", index=False, encoding='utf-8-sig')
print(f"✅ '마리수' 열 추가 완료! 총합: {df['마리수'].sum()}마리 → '서대문구_면적비율_마리수추가.csv' 저장됨")


✅ '마리수' 열 추가 완료! 총합: 19937마리 → '서대문구_면적비율_마리수추가.csv' 저장됨
